In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import spacy
from IPython.display import display

from src.pipelinelib.querying import Parser, Queryable

folder = os.path.join(os.getcwd(), "data", "transcripts")
files = [os.path.join(root, f) for root, _, files in os.walk(folder)
         for f in files if f.endswith(".docx")]


nlp = spacy.load("de_core_news_sm", disable=["ner", "parser"])
parser = Parser(
    nlp=nlp, metadata_path="./data/transcripts/Kopie von Transkriptionspaare_Daten.xls")

parser.read_from_files(files)
display(parser.frame)


=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 81_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 87_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 182_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 138_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 47_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 105_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 29_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 27_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 58_T1_IM_F

,document_id,paragraph_id,sentence_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,0,0,81,A,W,False,None,Ich dachte wir reden darüber wie wir die erste...
1,0,1,1,81,B,M,False,None,Erste Wohnung?
2,0,2,2,81,A,W,False,None,"Ja, ich dachte das ist ein nettes Thema, oder?"
3,0,3,3,81,B,M,False,None,Ja.
4,0,4,4,81,A,W,False,None,"Ich fand das süß, wie du erst angefangen hast ..."
...,...,...,...,...,...,...,...,...,...
131,9,94,131,60,B,M,True,W,Hab ich ein Schild auf dem Rücken?
132,9,94,132,60,B,M,True,W,Ladet eure Probleme bei mir ab.
133,9,95,133,60,A,W,True,W,Das sind keine Probleme.
134,9,96,134,60,B,M,True,W,Kümmerst du dich drum.


In [21]:
from src.pipelinelib.querying import Parser, Queryable
from src.pipelinelib.text_body import TextBody
from src.pipelinelib.pipeline import Pipeline
queryable = Queryable.from_parser(parser)

In [153]:
from src.sigmund.features import talk_turn as ftalkturn
from src.sigmund.preprocessing import words as pwords
from src.sigmund import adapter
from src.sigmund.extensions import LEMMATIZED, STEMMED, TOKENS, TALKTURN
#from src.sigmund.features import agreement_score as fagree

pipeline = Pipeline(queryable = queryable, empty_pipeline=True) \
    .add_component(pwords.Tokenizer())\
    .add_component(ftalkturn.TalkTurnExtractor()) 

#    .add_component(pwords.Tokenizer())\
#    .add_component(adapter.Adapter(old = TOKENS, new = TALKTURN))\
#    .add_component(ftalkturn.TalkTurnExtractor())  
#.add_component(fagree.AgreementScoreExtractor(liwc_dict_path))


In [156]:
test = pipeline.execute()

#for doc in preprocessed_depr:
#    print('TT',doc[0]._.talkturn, 'AS',doc[0]._.agreementscore)
#for doc in preprocessed_non_depr:
#    print('TT',doc[0]._.talkturn, 'AS',doc[0]._.agreementscore)

=== Starting pipeline with ['Tokenizer', 'TalkTurnExtractor'] ===
Executing Tokenizer
=== Queryable is executing on Sentence level, query = '' ===
Executing TalkTurnExtractor
=== Queryable is executing on Paragraph level, query = '' ===


,document_id,TalkTurn
0,0.0,0.57
1,1.0,0.63
2,2.0,0.57
3,3.0,0.46
4,4.0,0.43
5,5.0,0.46
6,6.0,0.37
7,7.0,0.50
8,8.0,0.58
9,9.0,0.62


=== Finished pipeline execution ===


In [157]:
display(test)

{<src.pipelinelib.extension.Extension at 0x7f750cd14d90>:      document_id  paragraph_id  sentence_id speaker  \
 0              0             0            0       A   
 1              0             1            1       B   
 2              0             2            2       A   
 3              0             3            3       B   
 4              0             4            4       A   
 ..           ...           ...          ...     ...   
 131            9            94          131       B   
 132            9            94          132       B   
 133            9            95          133       A   
 134            9            96          134       B   
 135            9            96          135       B   
 
                                                   text  
 0    [Ich, dachte, wir, reden, darüber, wie, wir, d...  
 1                                     [Erste, Wohnung]  
 2    [Ja, ich, dachte, das, ist, ein, nettes, Thema...  
 3                                   

In [ ]:
document_df = queryable.execute(level=TextBody.DOCUMENT)
display(document_df)

In [ ]:
paragraph_df = queryable.execute(level=TextBody.PARAGRAPH)
display(paragraph_df)

In [ ]:
sentence_df = queryable.execute(level=TextBody.SENTENCE)
display(sentence_df)

queryable.execute(level=TextBody.SENTENCE)

In [ ]:
pgs = queryable.by_couple_id(couple_id=27) \
    .is_depressed(d=False) \
    .execute(level=TextBody.DOCUMENT)
display(pgs)